In [1]:
from six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

import pymysql
import pydotplus
import pandas as pd
import sqlalchemy


In [9]:
#https://www.dataquest.io/blog/sql-insert-tutorial/
"""
def createDb():
    mydb = pymysql.connect(host='localhost', user='root', password='')
    if (mydb):
        print("Connection Successful")
    else:
        print("Connection Fail")
    cur=mydb.cursor()
    cur.execute("create database bus")
#createDb() 
"""

def connectDbCreateTableAndInsertData():
    connection = pymysql.connect(host='localhost', user='root', password='', db='bus')
    #This will allow us to execute the SQL query once we’ve written it.
    cursor = connection.cursor()
    sql1 = "CREATE TABLE IF NOT EXISTS `moneyback` (`id` int(10) NOT NULL,`fault` varchar(40) DEFAULT NULL,`user_type_golden_normal` varchar(40) DEFAULT NULL,`cancel_time` varchar(40) DEFAULT NULL,`payment_method` varchar(40) DEFAULT NULL,`money_back` varchar(40) DEFAULT NULL) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;"

    # Execute the query
    cursor.execute(sql1)

    
    # Create a new record
    sql = "INSERT INTO `moneyback` (`id`, `fault`,`user_type_golden_normal`,`cancel_time`,`payment_method`,`money_back`) VALUES (%s,%s,%s,%s,%s,%s)"
    
    cursor.execute(sql,(1,'ufault','golden','withinlimit','online','yes'))
    cursor.execute(sql,(2,'cfault','normal','exceed','cash','yes'))
    cursor.execute(sql,(3,'nofault','normal','withinlimit','online','yes'))
    cursor.execute(sql,(4,'ufault','normal','withinlimit','online','no'))
    cursor.execute(sql,(5,'nofault','golden','withinlimit','cash','yes'))
    cursor.execute(sql,(6,'nofault','golden','withinlimit','online','yes'))
    cursor.execute(sql,(7,'ufault','normal','exceed','online','no'))
    cursor.execute(sql,(8,'nofault','normal','withinlimit','cash','yes'))
    cursor.execute(sql,(9,'ufault','golden','exceed','online','no'))
    cursor.execute(sql,(10,'cfault','normal','withinlimit','cash','yes'))
    cursor.execute(sql,(11,'nofault','normal','exceed','online','no'))
    cursor.execute(sql,(12,'nofault','golden','exceed','cash','yes'))
    cursor.execute(sql,(13,'ufault','normal','withinlimit','online','no'))
    cursor.execute(sql,(14,'cfault','normal','exceed','online','no'))
    cursor.execute(sql,(15,'nofault','golden','withinlimit','cash','yes'))
    cursor.execute(sql,(16,'cfault','normal','exceed','online','no'))
    cursor.execute(sql,(17,'nofault','golden','withinlimit','online','yes'))
    cursor.execute(sql,(18,'cfault','golden','exceed','cash','no'))
    
    connection.commit()
    
    
def connectDbReadData():
    
    engine = sqlalchemy.create_engine('mysql+pymysql://root:@localhost:3306/bus')
    df = pd.read_sql_table('moneyback',engine, columns=['fault','user_type_golden_normal','cancel_time',
                                                        'payment_method','money_back'])
    return df



def convertWordToNumber(p):
    labelencoder_X = LabelEncoder()
    p = p.apply(LabelEncoder().fit_transform)
    return p


def train_using_entropy(X, y): 
  
    # Decision tree with entropy 
    dt_clf = DecisionTreeClassifier(criterion="entropy")
    dt_clf.fit(X, y)

    return dt_clf


"""Function to make predictions"""  
def prediction(X_test, clf_object): 
  
    # Predicton on test with giniIndex 
    y_pred = clf_object.predict(X_test) 
    print("Predicted values:") 
    print(y_pred) 
    return y_pred 
      
    
    
""" Function to calculate accuracy"""  
def cal_accuracy(y_test, y_pred): 
      
    print("Confusion Matrix: ", 
        confusion_matrix(y_test, y_pred)) 
      
    print ("Accuracy : ", 
    accuracy_score(y_test,y_pred)*100) 
      
    print("Report : ", 
    classification_report(y_test, y_pred)) 
    
    
"""Display tree"""
def showTree(dt_clf, feature):
    dot_data = StringIO()
    export_graphviz(dt_clf, out_file=dot_data, filled=True, rounded=True, special_characters=True,
                    feature_names=feature)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    graph.write_png('treee1234.png')

In [10]:
# Driver code 
def main(): 
      
    """
    Task:
    1. connect database
    2. create table
    3. insert data
    """
    ##connectDbCreateTableAndInsertData()
    
    
    #retrive data from database
    df = connectDbReadData()
    
    #print(df.head(10))
    
    p = df[['fault', 'user_type_golden_normal', 'cancel_time', 'payment_method','money_back']]

    feature = ['fault', 'user_type_golden_normal', 'cancel_time', 'payment_method']
    
    #use lebel encoder
    p = convertWordToNumber(p)
    ##print(p.head(10))
    
    X = p[['fault', 'user_type_golden_normal', 'cancel_time', 'payment_method']]
    y = p['money_back']
    
    # train using entropy
    dt_clf = train_using_entropy(X,y) 
        
    # X, y test data
    y_test = y[15:17]
    X_test = X[15:17]
    
    
    print("Results Using Entropy:") 
    # Prediction using entropy 
    y_pred_entropy = prediction(X_test, dt_clf) 
    cal_accuracy(y_test, y_pred_entropy) 
    
    # Visualize tree
    showTree(dt_clf, feature)
    
    
      
# Calling main function 
if __name__=="__main__": 
    main() 

Results Using Entropy:
Predicted values:
[0 1]
Confusion Matrix:  [[1 0]
 [0 1]]
Accuracy :  100.0
Report :                precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

